# Đồ Án Cuối Kì Môn Khoa Học Dữ Liệu

**Giới thiệu thành viên**

- *Họ tên:* **Trần Ngọc Tịnh**       
  *MSSV:* **18120597**
  
  
  
  
- *Họ tên:* **Nguyễn Ngọc Năng Toàn**       
  *MSSV:* **18120600**

## Import 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

# You can also import other things ...
# YOUR CODE HERE (OPTION)

## Đọc dữ liệu từ file csv